## 4.2 행렬 곱

In [1]:
import torch

In [ ]:
# x, y 토치 텐서 만들기

x = torch.FloatTensor([[1, 2],
                      [3, 4],
                      [5, 6]])
y = torch.FloatTensor([[1, 2],
                      [1, 2]])

print(x.size(), y.size())

torch.Size([3, 2]) torch.Size([2, 2])


In [ ]:
# x, y 토치 곱 연산

z = torch.matmul(x, y)
print(z.size())

# 기대한 값과 일치

torch.Size([3, 2])


### 1. 배치 행렬 곱

딥러닝을 수행할 때 보통 여러 샘플을 동시에 병렬 계산한다. 따라서 행렬 솝 연산의 경우에도 여러 곱셈을 동시에 진행할 수 있어야 한다.

In [ ]:
# bmm 함수 사용

x = torch.FloatTensor(3,3,2)
y = torch.FloatTensor(3,2,3)

z = torch.bmm(x, y)
print(z.size()) 

# torch.Size([3, 3, 3]) 3 x 3 크기의 행렬이 3개 만들어 진다.

torch.Size([3, 3, 3])


## 4.4 선형 계층

### 1. 직접 구현하기

선형 계층은 행렬 곱 연산과 브로드캐스팅 덧셈 연산으로 이루어져 있습니다. 선형 계층의 파라미터 행렬 W가 행렬 곱 연산에 활용될 것이고, 파라미터 벡터 b가 브로드캐스팅 것셈 연산에 활용될 것입니다. 앞에서 배운 파이토치 연산 방법이므로 그대로 다시 구현해보록 한다.

In [ ]:
# W, b 만들기

W = torch.FloatTensor([[1,2],
                       [3,4],
                       [5,6]]) # 3,2 크기의 행렬

b = torch.FloatTensor([2,2]) # 2개의 요소를 갖는 벡터 b

In [ ]:
# 선형 계층 함수를 구성하기

def linear(x, W, b):
    y = torch.matmul(x, W) + b

    return y

In [ ]:
# 선형 계층 계산

x = torch.FloatTensor(4,3)

y = linear(x, W, b)
print(y.size())

# 하지만 이 방법은 파이토치 입장에서 제대로 된 계층으로 취급되지 않습니다.
# 제대로 계층을 만드는 방법도 살펴본다.

torch.Size([4, 2])


### 2. torch.nn.Module 클래스 상속 받기

토치에는 nn(neural networks)패키지가 있고 내부에는 미리 정의된 많은 신경망들이 있다. 그리고 그 신경망들은 torch.nn.Module이라는 추상 클래스를 상속받아 정의되어 있다. 바로 이 추상 클래스를 상속받아 선형 계층을 구현할 수 있다.

In [ ]:
import torch.nn as nn

- 그리고 nn.Module을 상속받은 MyLunear라는 클래스를 정의한다. nn.Module을 상속받은 클래스는 보통 2개의 메서드, __init__과 forward를 오버라이드(override)합니다.

- __init__ 함수는 계층 내부에서 필요한 변수를 미리 선언하고 있으며 심지어 또 다른 계층(nn.Module을 상속받은 클래스의 객체)을 소유하도록 할 수도 있다. forward 함수는 계층을 통과하는데 필요한 계산 수행을 한다.



In [ ]:
class MyLinear(nn.Module):

    def __init__(self, input_dim=3, output_dim=2):
        self.input_dim = input_dim
        self.output_dim = output_dim

        super().__init__()

        self.W = torch.FloatTensor(input_dim, output_dim)
        self.b = torch.FloatTensor(output_dim)

    # You should override 'forward' method to implement detail.
    # The input argument and outputs can be desigend as you wish

    def forward(self, x):
        # |x| = (batch_size, imput_dim)
        y = torch.matmul(x, self.W) + self.b
        # |y| = (batch_size, imput_dim) * (input_dim, output_dim) = (batch_size, output_dim)

        return y

- forward 함수 내부에 계산되고 있는 텐서들의 모양을 주석으로 적어놓은 것을 볼 수 있다. 이렇게 주석을 통해 텐서의 모양을 미리 적어놓으면 추후에 코드를 파악하거나 디버깅을 할 때 훨씬 편리합니다. 앞에서와 같이 정의된 클래스를 이제 생성하여 사용할 수 있습니다.

In [ ]:
linear = MyLinear(3,2)

y = linear(x)

- 여기에서 즁요한 점은 forward 함수를 따로 호출하지 않고 객체명에 바로 괄호를 열어 텐서 x를 인수로 넘겨주었다는 것이다. 이처럼 nn.Module의 상속받은 객체는 __call__ 함수와 forward가 매핑되어 있어서 forward를 직접 부를 필요가 없습니다.

- forward 호출 앞뒤로 추가적으로 호출하는 함수가 파이토치 내부에 따로 있기 때문에 사용자가 직접 forward 함수를 호출한느 것을 권장하지 않습니다.

- 여기까지 nn.Module을 상속받아 선형 계층을 구성하여 보았습니다. 하지만 이 방법도 아직 제대로 된 방법은 아니다. 물론 파이토치 입장에서 MyLinear라는 클래스의 계층으로 인식하고 계산도 수행하지만 내부에 학습할 수 있는 파라미터는 없는 것으로 인식하기 때문입니다. 예를 들어 다음 코들를 실행하면 아무것도 출력되지 않는다.

In [ ]:
for p in linear.parameters():
    print(p)

### 3. 올바른 방법 : nn.Parameter 활용하기

제대로 된 방법은 W와 b를 파이토치에서 학습이 가능하도록 인식할 수 있는 파라미터로 만들어주어야 하는데 torch.nn.parameter 클래스를 활용하면 된다.

In [ ]:
class MyLinear(nn.Module):

    def __init__(self, input_dim=3, output_dim=2):
        self.input_dim = input_dim
        self.output_dim = output_dim

        super().__init__()

        self.W = nn.Parameter(torch.FloatTensor(input_dim, output_dim))
        self.b = nn.Parameter(torch.FloatTensor(output_dim))

    # You should override 'forward' method to implement detail.
    # The input argument and outputs can be desigend as you wish

    def forward(self, x):
        # |x| = (batch_size, imput_dim)
        y = torch.matmul(x, self.W) + self.b
        # |y| = (batch_size, imput_dim) * (input_dim, output_dim) = (batch_size, output_dim)

        return y

In [ ]:
linear = MyLinear(3,2)

y = linear(x)

In [ ]:
for p in linear.parameters():
    print(p)

Parameter containing:
tensor([[1.1820e-34, 0.0000e+00],
        [6.4460e-44, 1.1210e-43],
        [1.3593e-43, 1.5975e-43]], requires_grad=True)
Parameter containing:
tensor([1.1820e-34, 0.0000e+00], requires_grad=True)


### 4.nn.Linear 활용하기

- 지금까지 사용한 복잡한 방법 말고는 torch.nn에 미리 정의된 선형 계층을 불러다 사용하면 매우 간단하다. 다음 코드는 nn.Linear를 통해 선형 계층을 활용한 모습이다.

In [ ]:
linear = nn.Linear(3,2)

y = linear(x)

for p in linear.parameters():
    print(p)

Parameter containing:
tensor([[ 0.3459, -0.3852,  0.1171],
        [ 0.3976, -0.0546,  0.1626]], requires_grad=True)
Parameter containing:
tensor([0.3395, 0.2670], requires_grad=True)


- 또한 앞서 말한 대로 nn.Module을 상속받아 정의한 나만의 계층 클래스는 내부의 nn.Module 하위 클래스를 소유할 수 있다.

In [ ]:
class MyLinear(nn.Module):

    def __init__(self, input_dim=3, output_dim=2):
        self.input_dim = input_dim
        self.output_dim = output_dim

        super().__init__()

        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        # |x| = (batch_size, input_dim)
        y = self.linear(x)
        # |y| = (batch_size, output_dim)

        return y

- 앞의 코드는 nn.Module을 상속받아 MyLinear 클래스를 정의하고 있는데, __init__ 함수 내부에는 nn.Linear를 선언하여 self.linear에 저장하는 모습을 보여주고 있다. 그리고 forward 함수에서는 self.linear에 텐서 x를 통과시킨다. 즉, 이 코드도 선형 계층을 구현한 것이라 볼 수 있다.

### 4. GPU사용하기

#### 1. Cuda 함수
- cuda란? 엔비디아 gpu에서 수행하는 병렬 연산을 프로그래밍 언어를 통해 구현할 수 있도록 하는 기술입니다.

In [1]:
import torch

x = torch.cuda.FloatTensor(2,2)
x

tensor([[0., 0.],
        [0., 0.]], device='cuda:0')

cuda: 뒤에 붙은 숫자는 GPU 디바이스의 인덱스를 의미합니다. 즉, 첫번째 디바이스인 0번 GPU를 의미한다. 앞선 방법 외 텐서의 cuda 함수를 통해 CPU 메모리 상에 선언된 텐서를 GPU로 복사하는 방법도 존재

In [2]:
x = torch.FloatTensor(2,2)
x

tensor([[11886912.,        0.],
        [       0.,        0.]])

In [3]:
x = x.cuda()
x

tensor([[11886912.,        0.],
        [       0.,        0.]], device='cuda:0')

In [4]:
x = x.cuda(device=1) # 만약 2번째 gpu가 없다면 오류가 아래처럼 오류가 뜬다

RuntimeError: ignored

cuda 함수는 텐서뿐만 아니라 nn.Module의 하위 클래스 객체에서도 똑같이 적용할 수 있다.

In [5]:
import torch.nn as nn

layer = nn.Linear(2,2)
layer.cuda(0)

# 여기에서 주의할 점은 텐서는 cuda 함수를 통해 원하는 디바이스로 복사가 되었지만, nn.Module 하위 클래스 객체의 경우 복사가 아닌 이동이 수행된다는 점이다.

Linear(in_features=2, out_features=2, bias=True)

#### 2. 서로 다른 장치 간 연산
- 서로 다른 장치에 있는 텐서 또는 nn.Module의 하위 클래스 객체끼리는 연산이 불가능하다. cpu와 gpu에 위치한 텐서들끼리 연산이 불가능할 뿐만 아니라 0번 gpu와 1번 gpu 사이의 연산도 불가능하다.

In [6]:
x = torch.FloatTensor(2,2)
x

tensor([[1.1887e+07, 0.0000e+00],
        [8.5479e-44, 4.4842e-44]])

In [7]:
x + x.cuda(0)

RuntimeError: ignored

#### 3. cpu 함수

In [9]:
x = torch.cuda.FloatTensor(2,2)

x

tensor([[1.1887e+07, 0.0000e+00],
        [8.5479e-44, 4.4842e-44]], device='cuda:0')

In [11]:
x = x.cpu()
x

tensor([[1.1887e+07, 0.0000e+00],
        [8.5479e-44, 4.4842e-44]])

#### 4. To함수

to 함수는 원하는 디바이스의 정보를 담은 객체를 인자로 받아, 함수 자신을 호출한 객체를 해당 디바이스로 복사시킵니다. 디바이스 정보를 담은 torch.device를 통해 생성할 수 있습니다.

In [13]:
cpu_device = torch.device('cpu')
gpu_device = torch.device('cuda:0')

In [16]:
x = torch.FloatTensor(2,2)

x = x.to(gpu_device)
x

tensor([[1.1891e+07, 0.0000e+00],
        [8.5479e-44, 4.4842e-44]], device='cuda:0')

#### 5. Device 속성
텐서는 device 속성을 가직 있어 해당 텐서가 위치한 디바이스를 쉽게 파악할 수 있다.

In [18]:
x = torch.cuda.FloatTensor(2,2)
x.device

device(type='cuda', index=0)

In [19]:
layer = nn.Linear(2,2)
next(layer.parameters()).device

device(type='cpu')

## 4.6 마치며

### 요약

1. 선형 계층
 - 선형 계층은 행렬의 곱과 벡터의 덧셈으로 이루어져 있음
 - 선형 계층 또한 함수이며 내부의 가중치 파라미터의 값에 따라 출력값이 달라짐
 - 선형 계층을 통해 선형 데이터에 대한 관계를 분석하거나 선형 함수를 근사계산할 수 있음

2. 선형 계층의 수식
 - $y = f(x) = x*W+b$ 

3. gpu 활용하기
 - to 함수를 통해 텐서나 모델을 원하는 gpu로 복사/이동시킬 수 있음
 - 텐서끼리의 연산은 같은 장치(디바이스)에서만 가능
